<a href="https://colab.research.google.com/github/gastan81/generative_ai/blob/main/0_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Streamlit
Streamlit lets you transform your data scripts into interactive dashboards and prototypes in minutes, without needing front-end coding knowledge. This means you can easily share insights with colleagues, showcase your data science work, or even build simple machine learning tools, all within the familiar Python environment.

---
## 1.&nbsp; Streamlit demo 🚀
We first need to install [streamlit](https://streamlit.io/) - as always, locally this is a one time thing, whereas on colab we need to do it each session.

In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


To run Streamlit on Colab, we'll have to set up a tunnel. If you're working locally, you can skip this step.     
Modified from [Source](https://colab.research.google.com/gist/thapecroth/67a69d840010ffcfe7523655808c5b92/streamlit-on-colab.ipynb).

In [ ]:
# code necessary for Colab only

import os
import time
from IPython.display import display, HTML
def tunnel_prep():
    for f in ('cloudflared-linux-amd64', 'logs.txt', 'nohup.out'):
        try:
            os.remove(f'/content/{f}')
            print(f"Deleted {f}")
        except FileNotFoundError:
            continue

    !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -q
    !chmod +x cloudflared-linux-amd64
    !nohup /content/cloudflared-linux-amd64 tunnel --url http://localhost:8501 &
    url = ""
    while not url:
        time.sleep(1)
        result = !grep -o 'https://.*\.trycloudflare.com' nohup.out | head -n 1
        if result:
            url = result[0]
    return display(HTML(f'Your tunnel URL <a href="{url}" target="_blank">{url}</a>'))

Here's an example of what you can produce with streamlit. It's so easy, just a few lines of python depending on what you want, and so many options!

- Locally you would write this script in a .py file and not a notebook (.ipynb).

- On colab, we can create a .py file by using the magic command `%%writefile` at the top of the cell. This command writes the cell content to a file, naming it 'app.py', or whatever else you choose, in this instance. Once saved, you can see 'app.py' in Colab's storage by clicking on the left-hand side folder icon.

In [ ]:
%%writefile app.py

import streamlit as st

# Title
st.title("Streamlit Demo")

# Markdown
st.markdown("""
This is a demo app showcasing a few of Streamlit's features.

Streamlit is a powerful Python library for creating web apps. It is easy to use and has a wide range of features, including:

* **Interactive widgets:** Streamlit makes it easy to create interactive widgets, such as sliders, dropdown menus, and radio buttons.
* **Charts and graphs:** Streamlit can generate a variety of charts and graphs, including line charts, bar charts, and pie charts.
* **Data display:** Streamlit can display data in a variety of ways, including tables, lists, and maps.
* **Deployment:** Streamlit apps can be deployed to Heroku with a single command.
""")

# Slider
slider_value = st.slider("Select a number:", 0, 100)
st.write(f"You selected: {slider_value}")

# Dropdown menu
dropdown_value = st.selectbox("Choose a color:", ["red", "green", "blue"])
st.write(f"You chose: {dropdown_value}")

# Radio buttons
radio_button_value = st.radio("Select a language:", ["English", "Spanish", "French"])
st.write(f"You selected: {radio_button_value}")

# Text area
text = st.text_area("Enter some text:")
if text:
    st.write(f"You entered: {text}")

# Button
if st.button("Click me!"):
    st.write("You clicked the button!")

# Chart
data = {"x": [1, 2, 3, 4, 5], "y": [6, 7, 2, 4, 5]}
st.line_chart(data, x="x")

# Map
map_data = [
    {"name": "New York", "lat": 40.7128, "lon": -74.0060},
    {"name": "Los Angeles", "lat": 34.0522, "lon": -118.2437},
    {"name": "Chicago", "lat": 41.8783, "lon": -87.6233},
]
st.map(map_data)

Writing app.py


To run streamlit apps **locally**. Open the command line and navigate to the folder where you've stored the .py file. Then, use the command `streamlit run app.py`. If you used a different name for the .py file, change `app.py` for the name you used

On **colab**, as we have to run streamlit through a tunnel. This is a little annoying for debugging, as every time you encounter a bug, you have to stop and reopen the tunnel. However, if you have a slower computer, or you simply wish to use Google's power so that your resources are free to do other things, it's very useful.

### Local

Open a terminal and run this code. Make sure your current working directory is the same as contains your `app.py` file, and that you've activated an environment in which `streamlit` is installed.
```
streamlit run app.py
```

### Colab

In [ ]:
tunnel_prep()

Deleted cloudflared-linux-amd64


In [ ]:
!streamlit run app.py &>/content/logs.txt &

## 2.&nbsp; RAG chatbot in streamlit ⭐️
We'll start by installing the same libraries and downloading the same files as in the previous notebooks.

In [ ]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu

# download saved vector database for Alice's Adventures in Wonderland
gdown --folder 1A8A9lhcUXUKRrtCe7rckMlQtgmfLZRQH

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 33.3 MB/s eta 0:00:00
Processing file 1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt index.faiss
Processing file 1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ index.pkl


Retrieving folder contents
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1h_lk4wTr12FAEaCS3eIJ4xsdcmnuIGmt
To: /content/faiss_index/index.faiss
100%|██████████| 421k/421k [00:00<00:00, 117MB/s]
Downloading...
From: https://drive.google.com/uc?id=1O0Jz2Lx5cZdpQM7S5uw6Kx9_OLm5DuSQ
To: /content/faiss_index/index.pkl
100%|██████████| 216k/216k [00:00<00:00, 91.8MB/s]
Download completed


In [ ]:
import os
from google.colab import userdata # we stored our access token as a colab secret

os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')

Now, let's proceed by creating the .py file for our rag chatbot.

We sourced the foundational code for our Streamlit basic chatbot from the [Streamlit documentation](https://docs.streamlit.io/knowledge-base/tutorials/build-conversational-apps).

In addition, we implemented [cache_resource](https://docs.streamlit.io/library/api-reference/performance/st.cache_resource) for both memory and LLM. Given that Streamlit reruns the entire script with each message input, relying solely on memory would result in data overwriting and a loss of conversational continuity. The inclusion of cache resource prevents Streamlit from creating a new memory instance on each run. This was also added to the LLM, enhancing speed and preventing its reload in every iteration.

In [ ]:
%%writefile rag_app.py

from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever
from langchain.chains.retrieval import create_retrieval_chain
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain
import streamlit as st

# llm
hf_model = "mistralai/Mistral-7B-Instruct-v0.3"
llm = HuggingFaceEndpoint(repo_id=hf_model)

# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "/content/"

embeddings = HuggingFaceEmbeddings(model_name=embedding_model,
                                   cache_folder=embeddings_folder)

# load Vector Database
# allow_dangerous_deserialization is needed. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine
vector_db = FAISS.load_local("/content/faiss_index", embeddings, allow_dangerous_deserialization=True)

# retriever
retriever = vector_db.as_retriever(search_kwargs={"k": 2})

# prompt
template = """You are a nice chatbot having a conversation with a human. Answer the question based only on the following context and previous conversation. Keep your answers short and succinct.

Previous conversation:
{chat_history}

Context to answer question:
{context}

New human question: {input}
Response:"""

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# bot with memory
@st.cache_resource
def init_bot():
    doc_retriever = create_history_aware_retriever(llm, retriever, prompt)
    doc_chain = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(doc_retriever, doc_chain)

rag_bot = init_bot()


##### streamlit #####

st.title("Chatier & chatier: conversations in Wonderland")

# Initialise chat history
# Chat history saves the previous messages to be displayed
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("Curious minds wanted!"):

    # Display user message in chat message container
    st.chat_message("human").markdown(prompt)

    # Add user message to chat history
    st.session_state.messages.append({"role": "human", "content": prompt})

    # Begin spinner before answering question so it's there for the duration
    with st.spinner("Going down the rabbithole for answers..."):

        # send question to chain to get answer
        answer = rag_bot.invoke({"input": prompt, "chat_history": st.session_state.messages, "context": retriever})

        # extract answer from dictionary returned by chain
        response = answer["answer"]

        # Display chatbot response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)

        # Add assistant response to chat history
        st.session_state.messages.append({"role": "assistant", "content":  response})

Overwriting rag_app.py


Now, let's see what we've made.

### Local

Run this code in a terminal, following the caveats as laid out previously.
```
streamlit run rag_app.py
```

### Colab

In [ ]:
tunnel_prep()

Your tunnel url https://performed-surf-jones-gdp.trycloudflare.com


In [ ]:
!streamlit run rag_app.py &>/content/logs.txt &